In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
from tqdm import tqdm_notebook

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/input/'
ste = pd.read_csv(paths + 'sales_train_evaluation.csv')

In [ ]:
def train_model(store, cat, model, id):

  ret = dict()
  ret['id'] = id

  paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/data/'
  data_name = store +'_'+ cat + '_' + 'data' +'.csv'

  pre_ste = pd.read_csv(paths + data_name)
  pre_ste.iloc[ :-28, -1] = ste[ste['id'] == id].iloc[:, 6:].T.iloc[-28 * 5:, :].values

  pre_ste['sales_mean_7'] = pre_ste['target'].shift(1).rolling(window=7).mean()
  pre_ste['sales_mean_14'] = pre_ste['target'].shift(1).rolling(window=14).mean()
  pre_ste['sales_mean_21'] = pre_ste['target'].shift(1).rolling(window=21).mean()
  pre_ste['sales_mean_28'] = pre_ste['target'].shift(1).rolling(window=28).mean()

  pre_ste['sales_std_7'] = pre_ste['target'].shift(1).rolling(window=7).std()
  pre_ste['sales_std_14'] = pre_ste['target'].shift(1).rolling(window=14).std()
  pre_ste['sales_std_21'] = pre_ste['target'].shift(1).rolling(window=21).std()
  pre_ste['sales_std_28'] = pre_ste['target'].shift(1).rolling(window=28).std()

  for i in range(28, 0, -1):
    test = pre_ste.iloc[-i , :].drop(labels = ['date', 'd', 'target']).values.reshape(1, -1)

    pred = model.predict(test)
    ret['F'+str(29 - i)] = pred[0]
    pre_ste.iloc[-i, -1] = pred

    pre_ste['sales_mean_7'] = pre_ste['target'].shift(1).rolling(window=7).mean()
    pre_ste['sales_mean_14'] = pre_ste['target'].shift(1).rolling(window=14).mean()
    pre_ste['sales_mean_21'] = pre_ste['target'].shift(1).rolling(window=21).mean()
    pre_ste['sales_mean_28'] = pre_ste['target'].shift(1).rolling(window=28).mean()

    pre_ste['sales_std_7'] = pre_ste['target'].shift(1).rolling(window=7).std()
    pre_ste['sales_std_14'] = pre_ste['target'].shift(1).rolling(window=14).std()
    pre_ste['sales_std_21'] = pre_ste['target'].shift(1).rolling(window=21).std()
    pre_ste['sales_std_28'] = pre_ste['target'].shift(1).rolling(window=28).std()

  return ret

In [ ]:
from sklearn.externals import joblib

paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/model/'

ID_li = ste['id'].values.tolist()
result_li = []

for ID in tqdm_notebook(ID_li):
  id_li = ID.split('_')
  cat, store = id_li[0], id_li[3] + "_" + id_li[4]
  model_name = store +'_'+ cat + '_' + 'model' +'.pkl'
  load_model = joblib.load(paths + model_name)

  result = train_model(store, cat, load_model, ID)
  result_li.append(result)

result = pd.DataFrame(result_li)

paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/output/'
result.to_csv(paths + 'result.csv', index=False)
result.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.875960,0.790542,0.790542,0.790542,0.790542,1.175046,1.119955,0.903073,0.865044,0.905034,0.923207,0.900178,1.135565,1.059788,0.876449,0.878589,0.888464,0.898366,0.883373,1.052593,1.157903,0.989540,0.891053,0.904443,0.904443,0.928286,1.154377,1.131759
1,HOBBIES_1_002_CA_1_evaluation,0.945711,0.871804,0.871804,0.871804,0.871804,1.156194,1.156194,0.891321,0.862156,0.955886,0.928406,0.928406,1.111317,1.111317,0.855616,0.859407,0.859407,0.874780,0.874780,1.001198,0.985879,1.001958,0.929127,0.930789,0.930789,0.895277,1.154377,1.154377
2,HOBBIES_1_003_CA_1_evaluation,0.945711,0.821934,0.821934,0.821934,0.817294,1.156194,1.041497,0.890458,0.818636,0.884962,0.877948,0.892925,1.124762,1.051413,0.889191,0.866328,0.871697,0.871697,0.871697,1.002279,1.102183,0.983554,0.960305,0.898654,0.891053,0.891053,1.133370,1.059933
3,HOBBIES_1_004_CA_1_evaluation,0.875960,0.790542,0.790542,0.790542,0.790542,1.134866,1.134866,0.903073,0.859263,0.905034,0.916623,0.894671,1.083643,1.083643,0.888572,0.885944,0.878589,0.910206,0.883373,1.086905,1.086905,0.986365,0.891053,0.891053,0.904443,0.904443,1.185182,1.185182
4,HOBBIES_1_005_CA_1_evaluation,0.921879,0.860411,0.860411,0.860411,0.847241,1.178455,1.178455,0.918061,0.839392,0.988510,0.977217,0.922976,1.132573,1.132573,0.842752,0.878977,0.859407,0.859407,0.854602,1.001198,0.985879,0.969765,0.936401,0.922700,0.948042,0.968878,1.154377,1.146859


# 최종 제출

In [ ]:
paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/input/'
ss = pd.read_csv(paths + 'sample_submission.csv')

submisson = np.concatenate([ste.iloc[:, -28:].values, result.iloc[:, -28:].values])

ss.iloc[:, 1:] = submisson
paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/output/'
ss.to_csv(paths + 'submisson.csv', index=False)
ss.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,0.0,0.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,1.0,1.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0
